In [ ]:
# ============================================
# 🔹 Step 0. 환경 설정
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

plt.rcParams["figure.figsize"] = (12, 6)
plt.style.use("seaborn-v0_8-whitegrid")

file_path = "problem_data_final.xlsx"

# ============================================
# 🔹 Step 1. 데이터 불러오기 및 전처리
# ============================================
reward = pd.read_excel(file_path, sheet_name="Reward")
funding = pd.read_excel(file_path, sheet_name="Funding")
trade = pd.read_excel(file_path, sheet_name="Trade")
ip = pd.read_excel(file_path, sheet_name="IP")
spec = pd.read_excel(file_path, sheet_name="Spec")

# timestamp 정리
for df in [reward, funding, trade]:
    df["ts"] = pd.to_datetime(df["ts"])
trade = trade.sort_values(["account_id", "ts"])

# ============================================
# 🔹 Step 2. 병합 (±1시간 윈도우로 확대)
# ============================================
funding = funding.sort_values(["account_id", "ts"])
reward = reward.sort_values(["account_id", "ts"])

merged = pd.merge_asof(
    trade, funding,
    by="account_id", on="ts",
    direction="nearest", tolerance=pd.Timedelta("1H"),
    suffixes=("", "_funding")
)
merged = pd.merge_asof(
    merged, reward,
    by="account_id", on="ts",
    direction="nearest", tolerance=pd.Timedelta("1H"),
    suffixes=("", "_reward")
)
merged = merged.merge(ip, on="account_id", how="left")

# 결측치 처리
merged["funding_fee"] = merged["funding_fee"].fillna(0)
merged["reward_amount"] = merged["reward_amount"].fillna(0)

# ============================================
# 🔹 Step 3. 피처 계산 (기준선 없이 순수 시계열 값)
# ============================================
market_price = merged.groupby("symbol")["price"].transform("mean")
merged["price_diff_pct"] = abs(merged["price"] - market_price) / market_price * 100  # %
merged["pnl_ratio"] = merged.groupby("account_id")["price"].pct_change() * 100
merged["funding_rate"] = merged["funding_fee"] / (merged["amount"] + 1e-6)
merged["bonus_ratio"] = merged["reward_amount"] / (merged["amount"] + 1e-6)

ip_count = ip.groupby("ip")["account_id"].nunique().reset_index(name="ip_count")
merged = merged.merge(ip_count, on="ip", how="left")

merged["date"] = merged["ts"].dt.floor("D")
trade_freq = (
    merged.groupby(["account_id", pd.Grouper(key="ts", freq="7D")])["ts"]
    .count()
    .reset_index(name="trade_freq_7d")
)
merged = pd.merge_asof(
    merged.sort_values(["account_id", "ts"]),
    trade_freq.sort_values(["account_id", "ts"]),
    by="account_id", on="ts",
    direction="backward"
)

# ============================================
# 🔹 Step 4. 그래프 함수 (기준선 제거)
# ============================================
def plot_user_features_simple(user_id, df=merged):
    """유저별 피처 시계열 그래프 (기준선 제거)"""
    user_df = df[df["account_id"] == user_id].sort_values("ts")
    if user_df.empty:
        print(f"[!] 데이터 없음: {user_id}")
        return

    features = [
        ("price_diff_pct", "시장가 대비 체결가 (%)"),
        ("trade_freq_7d", "주간 거래 빈도"),
        ("pnl_ratio", "거래당 수익률 (%)"),
        ("funding_rate", "펀딩피 수익률"),
        ("bonus_ratio", "보너스 비율"),
        ("ip_count", "IP별 계정 수")
    ]

    fig, axes = plt.subplots(len(features), 1, figsize=(12, 3 * len(features)), sharex=True)

    for ax, (col, label) in zip(axes, features):
        ax.plot(user_df["ts"], user_df[col], marker="o", color="C0")
        ax.set_ylabel(label)
        ax.legend([label], loc="upper right")

    axes[-1].set_xlabel("시간 (ts)")
    plt.suptitle(f"유저별 핵심 피처 시계열 — {user_id}", fontsize=16)
    plt.tight_layout()
    plt.show()

# ============================================
# 🔹 Step 5. 실행
# ============================================
if merged.empty:
    print("⚠️ merged 데이터가 비어 있습니다. tolerance를 늘리거나 timestamp를 확인하세요.")
else:
    user_list = merged["account_id"].unique()
    print("✅ 그래프 생성 대상 유저:", user_list[:5])
    example_user = user_list[0]
    plot_user_features_simple(example_user)

# 여러 유저를 반복적으로 확인하려면:
# for uid in merged["account_id"].unique()[:5]:
#     plot_user_features_simple(uid)
